# 梯度检验

检验反向传播是否正确

In [2]:
# 导入必需的库

import numpy as np
from testCases import *
from gc_utils import sigmoid, relu, dictionary_to_vector, vector_to_dictionary, gradients_to_vector

## 数学中梯度的定义

$$ \frac{\partial J}{\partial \theta} = \lim_{\varepsilon \to 0} \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon} \tag{1}$$

## 1 维的梯度检验

假设我们有一个简单的1维线性函数$J(\theta) = \theta x$。这个函数（这个模型）只有一个参数$\theta$；$x$是输入。

下面我们会用代码来计算出$J(.)$（用前向传播计算出成本）然后计算出$\frac{\partial J}{\partial \theta}$（用反向传播计算出梯度）。

最后我们用梯度检验来证明反向传播计算出来的梯度是正确的。

![](./images/1Dgrad_kiank.png)

In [3]:
# 前向传播

def forward_propagation(x, theta):
    return np.dot(theta, x)

In [4]:
# 反向传播
# theta * x 的导数就是 x

def backward_propagation(x, theta):
    return x

# 梯度检验步骤

## 主要步骤 1
我们先通过下面5小步来利用前向传播计算出梯度，这个梯度我们用gradapprox表示。
1. $\theta^{+} = \theta + \varepsilon$
2. $\theta^{-} = \theta - \varepsilon$
3. $J^{+} = J(\theta^{+})$
4. $J^{-} = J(\theta^{-})$
5. $gradapprox = \frac{J^{+} - J^{-}}{2  \varepsilon}$

## 主要步骤 2
利用上面的反向传播也计算出一个梯度，这个梯度我们用grad表示

## 主要步骤 3
我们用下面的公式来计算gradapprox和grad这两个梯度相差多远。

$$ difference = \frac {\mid\mid grad - gradapprox \mid\mid_2}{\mid\mid grad \mid\mid_2 + \mid\mid gradapprox \mid\mid_2} \tag{2}$$   

如果两个梯度相差小于$10^{-7}$，那么说明这两个梯度很接近，说明你的反向传播是正确的；否则反向传播里面有问题。


In [5]:
def gradient_check(x, theta, epsilon=1e-7):
    # 利用前向传播计算梯度
    thetaplus = theta + epsilon
    thetapminus = theta - epsilon
    J_plus = forward_propagation(x, thetaplus)
    J_minus = forward_propagation(x, thetapminus)
    gradapprox = (J_plus - J_minus) / (2 * epsilon)

    # 利用反向传播计算梯度
    grad = backward_propagation(x, theta)

    # 对比梯度
    numerator = np.linalg.norm(grad - gradapprox)
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)
    difference = numerator / denominator

    if difference < 1e-7:
        print("反向传播是正确的")
    else:
        print("反向传播是错误的")

    return difference

In [6]:
x, theta = 2, 4
difference = gradient_check(x, theta)
print("difference = " + str(difference))


反向传播是正确的
difference = 2.919335883291695e-10


# 多维梯度检验

![](./images/NDgrad_kiank.png)



In [7]:
# 多维的前向传播

def forward_propagation_n(X, Y, parameters):
    m = X.shape[1]
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]

    # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)

    logprobs = np.multiply(-np.log(A3), Y) + np.multiply(-np.log(1 - A3), 1 - Y)
    cost = 1. / m * np.sum(logprobs)

    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)

    return cost, cache

In [11]:
# 多维的反向传播

def backward_propagation_n(X, Y, cache):
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache

    dZ3 = A3 - Y
    dW3 = 1. / m * np.dot(dZ3, A2.T)
    db3 = 1. / m * np.sum(dZ3, axis=1, keepdims=True)

    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1. / m * np.dot(dZ2, A1.T)
    db2 = 1. / m * np.sum(dZ2, axis=1, keepdims=True)

    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1. / m * np.dot(dZ1, X.T)
    db1 = 1. / m * np.sum(dZ1, axis=1, keepdims=True)

    return {"dZ3": dZ3, "dW3": dW3, "db3": db3,
            "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
            "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}

像1维检验时一样，我们还是使用下面的这个公式:

$$ \frac{\partial J}{\partial \theta} = \lim_{\varepsilon \to 0} \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon} \tag{1}$$

In [9]:
def gradient_check_n(parameters, gradients, X, Y, epsilon=1e-7):
    parameters_values, _ = dictionary_to_vector(parameters)
    grad = gradients_to_vector(gradients)
    num_parameters = parameters_values.shape[0]
    J_plus = np.zeros((num_parameters, 1))
    J_minus = np.zeros((num_parameters, 1))
    gradapprox = np.zeros((num_parameters, 1))

    # 计算gradapprox
    for i in range(num_parameters):
        thetaplus = np.copy(parameters_values)
        thetaplus[i][0] = thetaplus[i][0] + epsilon
        J_plus[i], _ = forward_propagation_n(X, Y, vector_to_dictionary(thetaplus))

        thetaminus = np.copy(parameters_values)
        thetaminus[i][0] = thetaminus[i][0] - epsilon
        J_minus[i], _ = forward_propagation_n(
        X, Y, vector_to_dictionary(thetaminus))

        gradapprox[i] = (J_plus[i] - J_minus[i]) / (2 * epsilon)

    numerator = np.linalg.norm(grad - gradapprox)                                
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)               
    difference = numerator / denominator        

    if difference > 2e-7:
        print("\033[93m" + "反向传播有问题! difference = " +
              str(difference) + "\033[0m")
    else:
        print("\033[92m" + "反向传播很完美! difference = " +
              str(difference) + "\033[0m")

    return difference

In [12]:
X, Y, parameters = gradient_check_n_test_case()

cost, cache = forward_propagation_n(X, Y, parameters)
gradients = backward_propagation_n(X, Y, cache)
difference = gradient_check_n(parameters, gradients, X, Y)


反向传播很完美! difference = 1.1885552035482147e-07
